# Comparing Total Gas Masses Between Analyses

This notebook measures gas masses for the samples we are using to verify our method, then compares them to the measurements from literature. While the focus of this work is the measurement of total galaxy cluster masses, comparing measured gas masses gives us an avenue to probe how similar our measurements of the baryon density of the clusters are to those from literature. Seeing as knowledge of the gas density profile feeds into the total gas mass, its important to know if we are consistent.

The XXL, LoCuSS, and Lovisari samples all have gas mass measurements associated with them which we will compare to, but the SDSSRM-XCS sample does not (such measurements will be presented in an upcoming paper).

## Import Statements

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity, Unit
from astropy.cosmology import LambdaCDM, WMAP9
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 120
from xga.samples import ClusterSample
from xga.imagetools.psf import rl_psf
from xga.sourcetools.density import inv_abel_fitted_model, ann_spectra_apec_norm
from xga.models import BetaProfile1D, DoubleBetaProfile1D, KingProfile1D, SimpleVikhlininDensity1D
from xga.exceptions import ModelNotAssociatedError

## Defining useful functions

In [2]:
# This function is used for getting axis limits for the the one to one comparison plots later
def find_lims(x_dat, y_dat, buffer=0.1):
    # A buffer of some percent (default 10) is added to the max and min values to make the plot 
    #  more easily readable
    lom = 1 - buffer
    him = 1 + buffer
    
    # Reading out the values without errors
    x_vals = x_dat[:, 0]
    y_vals = y_dat[:, 0]
    
    # Depending on whether the input data has + and - errors, or just a standard deviation, depends how
    #  we find maximum and minimum values
    if x_dat.shape[1] == 3:
        # In this case upper and lower errors are present
        lb = x_vals-x_dat[:, 1]
        # Make sure that we don't count any NaN values, and don't count any negative values
        #  The lower errors are subtracted from the measurements, and upper values added to them
        #  Then max and mins are found 
        x_lims = [np.nanmin(lb[np.where(lb>0)[0]]), np.nanmax(x_vals+x_dat[:, 2])]
    elif x_dat.shape[1] == 2:
        # The behaviour is largely the same as above, but for symmetrical errors
        lb = x_vals-x_dat[:, 1]
        x_lims = [np.nanmin(lb[np.where(lb>0)[0]]), np.nanmax(x_vals+x_dat[:, 1])]
    
    if y_dat.shape[1] == 3:
        lb = y_vals-y_dat[:, 1]
        y_lims = [np.nanmin(lb[np.where(lb>0)[0]]), np.nanmax(y_vals+y_dat[:, 2])]
    elif y_dat.shape[1] == 2:
        lb = y_vals-y_dat[:, 1]
        y_lims = [np.nanmin(lb[np.where(lb>0)[0]]), np.nanmax(y_vals+y_dat[:, 1])]
    
    # Then find the minimum and maximum values from the min and max x and y data, and multiply by the buffer
    lims = Quantity([lom*min([x_lims[0], y_lims[0]]), him*max([x_lims[1], y_lims[1]])])
    
    # Return the limits for the square like for like comparison plot
    return lims

## Reading colour configuration file

In [3]:
samp_colours = pd.read_csv("samp_plot_colours.csv")

lov_colour = samp_colours[samp_colours['samp_name'] == 'lovisari'].iloc[0]['samp_colour']
xxl_colour = samp_colours[samp_colours['samp_name'] == 'xxl'].iloc[0]['samp_colour']
loc_colour = samp_colours[samp_colours['samp_name'] == 'locuss'].iloc[0]['samp_colour']

## Setting up cosmology

The LoCuSS and Lovisari analyses use the same concordance cosmology, but the XXL analysis of their bright cluster sample uses the WMAP9 results, and in case we wish to change anything for an individual sample later we define three separate cosmology objects to pass into our samples.

In [4]:
xxl_cosmo = WMAP9
locuss_cosmo = LambdaCDM(70, 0.3, 0.7)
lovisari_cosmo = LambdaCDM(70, 0.3, 0.7)

## Reading in Sample Files and Declaring XGA ClusterSamples

$\color{red}{\text{NEED TO MENTION/PUBLISH THE OBSIDS THAT WE EXCLUDE FROM USE DUE TO DATA PROBLEMS LIKE FLARING}}$

This subsection involves reading in the sample files of the four test samples (described in [the sample properties notebook](sample_properties.ipynb)), then setting up separate XGA ClusterSample instances (see [the documentation](https://xga.readthedocs.io/en/latest/notebooks/tutorials/sources_samples.html) for an introduction to XGA source and sample objects.

We impose an additional cleaning step on each sample, where we make sure that (for each XMM observation initially associated with a source) at least 70% of a cluster's $R_{500}$ falls on that observation - if this requirement is not met then the observation is excluded. These requirements are set with the `clean_obs=True`, `clean_obs_reg='r500'`, and `clean_obs_threshold=0.7` arguments when a ClusterSample instance is declared.

### XXL-100-GC

This is the sample of the brightest clusters in the XXL survey. It contains temperature and luminosity measurements, and will be a useful external comparison for XGA results, though the shallow XXL data may prove a challenge.

In [5]:
xxlgc100 = pd.read_csv("sample_files/xxl_gc100.csv")

# Limit the comparison to clusters with a flag of 0 - meaning it is in the main sample of 100 clusters
xxlgc100 = xxlgc100[xxlgc100['Flag'] == 0]

# Excluding a specific cluster which was excluded in the XXL analysis
xxlgc100 = xxlgc100[xxlgc100['XLSSC'] != 504]

In [ ]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = xxlgc100['ra'].values
dec = xxlgc100['dec'].values
z = xxlgc100['z'].values
n = xxlgc100['name'].values
r500 = Quantity(xxlgc100['r500MT'].values, 'Mpc')

# Declaring the actual ClusterSample instance for the XXL sample
# This is the only sample whose original analysis used the WMAP9 cosmology
xxl_srcs = ClusterSample(ra, dec, z, n, r500=r500, cosmology=xxl_cosmo, load_fits=True, use_peak=False, 
                         clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)

Declaring BaseSource Sample:  44%|███████████████████▌                        | 44/99 [02:39<03:43,  4.06s/it]

### LoCuSS High-$L_{\rm{X}}$

The LoCuSS High-$L_{\rm{X}}$ sample was selected from ROSAT for its high luminosity clusters, and will again be a useful comparison as testing against various different analyses is beneficial in establishing the veracity of our new measurements.

In [ ]:
locuss = pd.read_csv("sample_files/locuss_highlx_clusters.csv", dtype={'chandra_id': str, 'xmm_obsid': str})

In [ ]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = locuss['ra'].values
dec = locuss['dec'].values
z = locuss['z'].values
n = locuss['name'].values
r500 = Quantity(locuss['r500'].values, 'kpc')
r2500 = Quantity(locuss['r2500'].values, 'kpc')


# Declaring the actual ClusterSample instance for the LoCuSS sample
locuss_srcs = ClusterSample(ra, dec, z, n, r500=r500, r2500=r2500, cosmology=locuss_cosmo, load_fits=True, 
                            use_peak=False, clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)

### Planck Selected with XMM follow-up

The Lovisari et al. sample was selected from Planck-eSZ, and will again be a useful comparison as testing against various different analyses is beneficial in establishing the veracity of our new measurements.

In [ ]:
lovisari = pd.read_csv("sample_files/lovisari_planck_clusters.csv")

In [ ]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = lovisari['planck_ra'].values
dec = lovisari['planck_dec'].values
z = lovisari['z'].values
n = lovisari['name'].values
r500 = Quantity(lovisari['r500'].values, 'kpc')

# Declaring the actual ClusterSample instance for the Lovisari sample
lovisari_srcs = ClusterSample(ra, dec, z, n, r500=r500, cosmology=lovisari_cosmo, load_fits=True, use_peak=False, 
                            clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)

## Running PSF Correction

In [ ]:
rl_psf(xxl_srcs)
rl_psf(locuss_srcs)
rl_psf(lovisari_srcs)

## Reading in literature data

### Gas masses from literature

In [ ]:
xxl_lit_gms = [xxlgc100[xxlgc100['name'] == n].iloc[0][['mg500', 'mg500_err']].values*1e+13 
               for n in xxl_srcs.names]
xxl_lit_gms = Quantity(xxl_lit_gms, 'Msun')

lov_lit_gms = [lovisari[lovisari['name'] == n].iloc[0][['mg500', 'mg500-', 'mg500+']].values*1e+13
               for n in lovisari_srcs.names]
lov_lit_gms = Quantity(lov_lit_gms, 'Msun')

loc_lit_gms = [locuss[locuss['name'] == n].iloc[0][['x_mg500', 'x_mg500_err']].values
               for n in locuss_srcs.names]
loc_lit_gms = Quantity(loc_lit_gms, 'Msun')

### Conversion Temperatures

In [ ]:
xxl_conv_txs = []
loc_conv_txs = []
lov_conv_txs = []

for src in xxl_srcs:
    xxl_conv_txs.append(xxlgc100[xxlgc100['name'] == src.name].iloc[0]['T300kpc'])
    
for src in locuss_srcs:
    loc_conv_txs.append(locuss[locuss['name'] == src.name].iloc[0]['x_Tx500ce'])

for src in lovisari_srcs:
    lov_conv_txs.append(lovisari[lovisari['name'] == src.name].iloc[0]['Tx'])

xxl_conv_txs = Quantity(xxl_conv_txs, 'keV')
loc_conv_txs = Quantity(loc_conv_txs, 'keV')
lov_conv_txs = Quantity(lov_conv_txs, 'keV')

## Gas Density Profile Generation and Comparison to Literature - from $S_{B}$ Profiles

### Beta and King Profiles - out to 1.3$R_{500}$

In [ ]:
sb_mod = BetaProfile1D()
sb_mod.info()
sb_mod.par_priors

In [ ]:
xxl_beta_13_dp = inv_abel_fitted_model(xxl_srcs, sb_mod, 'mcmc', xxl_srcs.r500*1.3, 
                                       conv_temp=xxl_conv_txs, conv_outer_radius=Quantity(300, 'kpc'))
lov_beta_13_dp = inv_abel_fitted_model(lovisari_srcs, sb_mod, 'mcmc', lovisari_srcs.r500*1.3, 
                                       conv_temp=lov_conv_txs)
loc_beta_13_dp = inv_abel_fitted_model(locuss_srcs, sb_mod, 'mcmc', locuss_srcs.r500*1.3, 
                                       conv_temp=loc_conv_txs)

In [ ]:
d_mod = KingProfile1D(y_unit=Unit("1 / cm^3"))
d_mod.info()
d_mod.par_priors

In [ ]:
with tqdm(desc="Fitting density model to XXL profiles", total=len(xxl_beta_13_dp)) as onwards:
    for dp in xxl_beta_13_dp:
        if dp is not None:
            dp.fit(d_mod, progress_bar=False)
        onwards.update(1)

with tqdm(desc="Fitting density model to Lovisari profiles", total=len(lov_beta_13_dp)) as onwards:
    for dp in lov_beta_13_dp:
        if dp is not None:
            dp.fit(d_mod, progress_bar=False)
        onwards.update(1)

with tqdm(desc="Fitting density model to LoCuSS profiles", total=len(loc_beta_13_dp)) as onwards:
    for dp in loc_beta_13_dp:
        if dp is not None:
            dp.fit(d_mod, progress_bar=False)
        onwards.update(1)


In [ ]:
xxl_beta_king_13_gm = []
with tqdm(desc="Calculating XXL gas masses", total=len(xxl_beta_13_dp)) as onwards:
    for dp in xxl_beta_13_dp:
        if dp is not None:
            gm = dp.gas_mass("king", xxl_srcs[dp.src_name].r500)[0]
            xxl_beta_king_13_gm.append(gm)
        else:
            xxl_beta_king_13_gm.append([np.NaN, np.NaN, np.NaN])
        onwards.update(1)

lov_beta_king_13_gm = []
with tqdm(desc="Calculating Lovisari gas masses", total=len(lov_beta_13_dp)) as onwards:
    for dp in lov_beta_13_dp:
        if dp is not None:
            gm = dp.gas_mass("king", lovisari_srcs[dp.src_name].r500)[0]
            lov_beta_king_13_gm.append(gm)
        else:
            lov_beta_king_13_gm.append([np.NaN, np.NaN, np.NaN])
        onwards.update(1)        
        
loc_beta_king_13_gm = []
with tqdm(desc="Calculating LoCuSS gas masses", total=len(loc_beta_13_dp)) as onwards:
    for dp in loc_beta_13_dp:
        if dp is not None:
            gm = dp.gas_mass("king", locuss_srcs[dp.src_name].r500)[0]
            loc_beta_king_13_gm.append(gm)
        else:
            loc_beta_king_13_gm.append([np.NaN, np.NaN, np.NaN])
        onwards.update(1)        
        
xxl_beta_king_13_gm = Quantity(xxl_beta_king_13_gm, 'Msun')
lov_beta_king_13_gm = Quantity(lov_beta_king_13_gm, 'Msun')
loc_beta_king_13_gm = Quantity(loc_beta_king_13_gm, 'Msun')

In [ ]:
# Setting the y-position and font size of the a, b, and c labels that are added below the subplots
sublabel_ypos = -0.115
sublabel_fsize = 14

# Set up the matplotlib figure with 3 columns, meaning three subplots
fig, ax_arr = plt.subplots(ncols=3, figsize=(16, 6))

# Iterating through the array of axes objects, setting up the ticks
for ax in ax_arr:
    ax.minorticks_on()
    ax.tick_params(which='both', top=True, right=True, direction='in')

# Reading out the first axis in the array of axes
ax = ax_arr[0]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
xxl_lims = find_lims(xxl_lit_gms, xxl_beta_king_13_gm).value

# Plotting a 1:1 line to help with comparisons
plt.plot(xxl_lims, xxl_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(xxl_lit_gms[:, 0].value, xxl_beta_king_13_gm[:, 0].value, xerr=xxl_lit_gms[:, 1].value, 
             yerr=[xxl_beta_king_13_gm[:, 1].value, xxl_beta_king_13_gm[:, 2].value], fmt='x', 
             color=xxl_colour, capsize=2, label="XXL-100-GC")

# # Labelling axes, setting labels etc.
ax.set_xlabel(r"XXL Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(xxl_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(xxl_lims)

# Add the a) label below this first plot
ax.text(0.5, sublabel_ypos, s='a)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

# Reading out the second axis in the array of axes
ax = ax_arr[1]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
lov_lims = find_lims(lov_lit_gms, lov_beta_king_13_gm).value

# Plotting a 1:1 line to help with comparisons
plt.plot(lov_lims, lov_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(lov_lit_gms[:, 0].value, lov_beta_king_13_gm[:, 0].value, 
             xerr=[lov_lit_gms[:, 1].value, lov_lit_gms[:, 2].value], 
             yerr=[lov_beta_king_13_gm[:, 1].value, lov_beta_king_13_gm[:, 2].value], fmt='x', 
             color=lov_colour, capsize=2, label="Lovisari et al.")

# Labelling axes, setting labels etc.
ax.set_xlabel(r"Lovisari Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(lov_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(lov_lims)
ax.text(0.5, sublabel_ypos, s='b)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

# Reading out the last axis in the array of axes
ax = ax_arr[2]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
loc_lims = find_lims(loc_lit_gms, loc_beta_king_13_gm).value

# Plotting a 1:1 line to help with comparisons
plt.plot(loc_lims, loc_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(loc_lit_gms[:, 0].value, loc_beta_king_13_gm[:, 0].value, 
             xerr=loc_lit_gms[:, 1].value, 
             yerr=[loc_beta_king_13_gm[:, 1].value, loc_beta_king_13_gm[:, 2].value], fmt='x', 
             color=loc_colour, capsize=2, label=r"LoCuSS High-$L_{\rm{X}}$")

# Labelling axes, setting labels etc.
ax.set_xlabel(r"LoCuSS Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(loc_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(loc_lims)
ax.text(0.5, sublabel_ypos, s='c)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

# Saving and plotting the resulting figure
plt.tight_layout()
# plt.savefig("non_paper_figures/xxl_lov_loc_beta_king_1.3r500_gm_comp.pdf")
plt.show()

### Double Beta and Simplified Vikhlinin Profiles - out to 1.3$R_{500}$

In [ ]:
sb_mod = DoubleBetaProfile1D()
sb_mod.info()

xxl_dblbeta_13_dp = inv_abel_fitted_model(xxl_srcs, sb_mod, 'mcmc', xxl_srcs.r500*1.3, 
                                       conv_temp=xxl_conv_txs, conv_outer_radius=Quantity(300, 'kpc'))
lov_dblbeta_13_dp = inv_abel_fitted_model(lovisari_srcs, sb_mod, 'mcmc', lovisari_srcs.r500*1.3, 
                                       conv_temp=lov_conv_txs)
loc_dblbeta_13_dp = inv_abel_fitted_model(locuss_srcs, sb_mod, 'mcmc', locuss_srcs.r500*1.3, 
                                       conv_temp=loc_conv_txs)

In [ ]:
d_mod = SimpleVikhlininDensity1D(y_unit=Unit("1 / cm^3"))
d_mod.info()
print(d_mod.par_priors)

with tqdm(desc="Fitting density model to XXL profiles", total=len(xxl_dblbeta_13_dp)) as onwards:
    for dp in xxl_dblbeta_13_dp:
        if dp is not None:
            dp.fit(d_mod, progress_bar=False)
        onwards.update(1)

with tqdm(desc="Fitting density model to Lovisari profiles", total=len(lov_dblbeta_13_dp)) as onwards:
    for dp in lov_dblbeta_13_dp:
        if dp is not None:
            dp.fit(d_mod, progress_bar=False)
        onwards.update(1)

with tqdm(desc="Fitting density model to LoCuSS profiles", total=len(loc_dblbeta_13_dp)) as onwards:
    for dp in loc_dblbeta_13_dp:
        if dp is not None:
            dp.fit(d_mod, progress_bar=False)
        onwards.update(1)


In [ ]:
xxl_dblbeta_svikh_13_gm = []
with tqdm(desc="Calculating XXL gas masses", total=len(xxl_dblbeta_13_dp)) as onwards:
    for dp in xxl_dblbeta_13_dp:
        if dp is not None:
            gm = dp.gas_mass("simple_vikhlinin_dens", xxl_srcs[dp.src_name].r500)[0]
            xxl_dblbeta_svikh_13_gm.append(gm)
        else:
            xxl_dblbeta_svikh_13_gm.append([np.NaN, np.NaN, np.NaN])
        onwards.update(1)

lov_dblbeta_svikh_13_gm = []
with tqdm(desc="Calculating Lovisari gas masses", total=len(lov_dblbeta_13_dp)) as onwards:
    for dp in lov_dblbeta_13_dp:
        if dp is not None:
            gm = dp.gas_mass("simple_vikhlinin_dens", lovisari_srcs[dp.src_name].r500)[0]
            lov_dblbeta_svikh_13_gm.append(gm)
        else:
            lov_dblbeta_svikh_13_gm.append([np.NaN, np.NaN, np.NaN])
        onwards.update(1)        
        
loc_dblbeta_svikh_13_gm = []
with tqdm(desc="Calculating LoCuSS gas masses", total=len(loc_dblbeta_13_dp)) as onwards:
    for dp in loc_dblbeta_13_dp:
        if dp is not None:
            gm = dp.gas_mass("simple_vikhlinin_dens", locuss_srcs[dp.src_name].r500)[0]
            loc_dblbeta_svikh_13_gm.append(gm)
        else:
            loc_dblbeta_svikh_13_gm.append([np.NaN, np.NaN, np.NaN])
        onwards.update(1)        
        
xxl_dblbeta_svikh_13_gm = Quantity(xxl_dblbeta_svikh_13_gm, 'Msun')
lov_dblbeta_svikh_13_gm = Quantity(lov_dblbeta_svikh_13_gm, 'Msun')
loc_dblbeta_svikh_13_gm = Quantity(loc_dblbeta_svikh_13_gm, 'Msun')

In [ ]:
# Setting the y-position and font size of the a, b, and c labels that are added below the subplots
sublabel_ypos = -0.115
sublabel_fsize = 14

# Set up the matplotlib figure with 3 columns, meaning three subplots
fig, ax_arr = plt.subplots(ncols=3, figsize=(16, 6))

# Iterating through the array of axes objects, setting up the ticks
for ax in ax_arr:
    ax.minorticks_on()
    ax.tick_params(which='both', top=True, right=True, direction='in')

# Reading out the first axis in the array of axes
ax = ax_arr[0]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
xxl_lims = find_lims(xxl_lit_gms, xxl_dblbeta_svikh_13_gm).value

# Plotting a 1:1 line to help with comparisons
plt.plot(xxl_lims, xxl_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(xxl_lit_gms[:, 0].value, xxl_dblbeta_svikh_13_gm[:, 0].value, xerr=xxl_lit_gms[:, 1].value, 
             yerr=[xxl_dblbeta_svikh_13_gm[:, 1].value, xxl_dblbeta_svikh_13_gm[:, 2].value], fmt='x', 
             color=xxl_colour, capsize=2, label="XXL-100-GC")

# # Labelling axes, setting labels etc.
ax.set_xlabel(r"XXL Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(xxl_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(xxl_lims)

# Add the a) label below this first plot
ax.text(0.5, sublabel_ypos, s='a)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

# Reading out the second axis in the array of axes
ax = ax_arr[1]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
lov_lims = find_lims(lov_lit_gms, lov_dblbeta_svikh_13_gm).value

# Plotting a 1:1 line to help with comparisons
plt.plot(lov_lims, lov_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(lov_lit_gms[:, 0].value, lov_dblbeta_svikh_13_gm[:, 0].value, 
             xerr=[lov_lit_gms[:, 1].value, lov_lit_gms[:, 2].value], 
             yerr=[lov_dblbeta_svikh_13_gm[:, 1].value, lov_dblbeta_svikh_13_gm[:, 2].value], fmt='x', 
             color=lov_colour, capsize=2, label="Lovisari et al.")

# Labelling axes, setting labels etc.
ax.set_xlabel(r"Lovisari Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(lov_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(lov_lims)
ax.text(0.5, sublabel_ypos, s='b)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

# Reading out the last axis in the array of axes
ax = ax_arr[2]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
loc_lims = find_lims(loc_lit_gms, loc_dblbeta_svikh_13_gm).value

# Plotting a 1:1 line to help with comparisons
plt.plot(loc_lims, loc_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(loc_lit_gms[:, 0].value, loc_dblbeta_svikh_13_gm[:, 0].value, 
             xerr=loc_lit_gms[:, 1].value, 
             yerr=[loc_dblbeta_svikh_13_gm[:, 1].value, loc_dblbeta_svikh_13_gm[:, 2].value], fmt='x', 
             color=loc_colour, capsize=2, label=r"LoCuSS High-$L_{\rm{X}}$")

# Labelling axes, setting labels etc.
ax.set_xlabel(r"LoCuSS Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(loc_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(loc_lims)
ax.text(0.5, sublabel_ypos, s='c)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

# Saving and plotting the resulting figure
plt.tight_layout()
plt.savefig("non_paper_figures/xxl_lov_loc_dblbeta_svikh_1.3r500_gm_comp.pdf")
plt.show()

## Gas Density Profile Generation and Comparison to Literature - from Annular Spectra

### Simplified Vikhlinin Profiles - out to $R_{500}$

In [ ]:
loc_annspec_10_dp = ann_spectra_apec_norm(locuss_srcs, locuss_srcs.r500, min_snr=30)
lov_annspec_10_dp = ann_spectra_apec_norm(lovisari_srcs, lovisari_srcs.r500, min_snr=30)

In [ ]:
dpath = "non_paper_figures/density_profiles/svikh_1r500_annspec/"
if not os.path.exists(dpath):
    os.makedirs(dpath+"lovisari")
    os.makedirs(dpath+"locuss") 

lov_svikh_10_annspec_gm = []
with tqdm(desc="Measuring Lovisari Gas Masses", total=len(lov_annspec_10_dp)) as onwards:
    for dp in lov_annspec_10_dp:
        if dp is not None:
            rel_src = lovisari_srcs[dp.src_name]
            # Fit a simplified Vikhlinin density profile
            dp.fit("simple_vikhlinin_dens", progress_bar=False)
            dp.savefig(dpath+"lovisari/{}.pdf".format(rel_src.name))
            try:
                # Measure a gas mass within R500
                gm = dp.gas_mass("simple_vikhlinin_dens", rel_src.r500)[0]
                lov_svikh_10_annspec_gm.append(gm)
            except ModelNotAssociatedError:
                lov_svikh_10_annspec_gm.append([np.NaN, np.NaN, np.NaN])
        else:
            lov_svikh_10_annspec_gm.append([np.NaN, np.NaN, np.NaN])
    
        onwards.update(1)

loc_svikh_10_annspec_gm = []
with tqdm(desc="Measuring LoCuSS Gas Masses", total=len(loc_annspec_10_dp)) as onwards:
    for dp in loc_annspec_10_dp:
        if dp is not None:
            rel_src = locuss_srcs[dp.src_name]
            # Fit a simplified Vikhlinin density profile
            dp.fit("simple_vikhlinin_dens", progress_bar=False)
            dp.savefig(dpath+"locuss/{}.pdf".format(rel_src.name))
            try:
                # Measure a gas mass within R500
                gm = dp.gas_mass("simple_vikhlinin_dens", rel_src.r500)[0]
                loc_svikh_10_annspec_gm.append(gm)
            except ModelNotAssociatedError:
                loc_svikh_10_annspec_gm.append([np.NaN, np.NaN, np.NaN])
        else:
            loc_svikh_10_annspec_gm.append([np.NaN, np.NaN, np.NaN])
    
        onwards.update(1)

lov_svikh_10_annspec_gm = Quantity(lov_svikh_10_annspec_gm, 'Msun').astype(float)
loc_svikh_10_annspec_gm = Quantity(loc_svikh_10_annspec_gm, 'Msun').astype(float)

In [ ]:
# Setting the y-position and font size of the a and b  labels that are added below the subplots
sublabel_ypos = -0.115
sublabel_fsize = 14

# Set up the matplotlib figure with 2 columns, meaning three subplots
fig, ax_arr = plt.subplots(ncols=2, figsize=(12, 6))

# Iterating through the array of axes objects, setting up the ticks
for ax in ax_arr:
    ax.minorticks_on()
    ax.tick_params(which='both', top=True, right=True, direction='in')

# Reading out the first axis in the array of axes
ax = ax_arr[0]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
lov_lims = find_lims(lov_lit_gms, lov_svikh_10_annspec_gm).value
lov_lims[1] = 5e+14

# Plotting a 1:1 line to help with comparisons
plt.plot(lov_lims, lov_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(lov_lit_gms[:, 0].value, lov_svikh_10_annspec_gm[:, 0].value, 
             xerr=lov_lit_gms[:, 1:].T.value, 
             yerr=lov_svikh_10_annspec_gm[:, 1:].T.value, fmt='x', 
             color=lov_colour, capsize=2, label="Lovisari et al.")

# Labelling axes, setting labels etc.
ax.set_xlabel(r"Lovisari Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(lov_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(lov_lims)
ax.text(0.5, sublabel_ypos, s='a)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

# Reading out the second axis in the array of axes
ax = ax_arr[1]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
loc_lims = find_lims(loc_lit_gms, loc_svikh_10_annspec_gm).value

# Plotting a 1:1 line to help with comparisons
plt.plot(loc_lims, loc_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(loc_lit_gms[:, 0].value, loc_svikh_10_annspec_gm[:, 0].value, 
             xerr=loc_lit_gms[:, 1].value, 
             yerr=loc_svikh_10_annspec_gm[:, 1:].T.value, fmt='x', 
             color=loc_colour, capsize=2, label=r"LoCuSS High-$L_{\rm{X}}$")

# Labelling axes, setting labels etc.
ax.set_xlabel(r"LoCuSS Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(loc_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(loc_lims)
ax.text(0.5, sublabel_ypos, s='b)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

plt.tight_layout()

# Saving and plotting the resulting figure
plt.savefig("non_paper_figures/lov_loc_svikh_1r500_annspec_gm_comp.pdf")
plt.show()

### Simplified Vikhlinin Profiles - out to 1.3$R_{500}$

In [ ]:
loc_annspec_13_dp = ann_spectra_apec_norm(locuss_srcs, locuss_srcs.r500*1.3, min_snr=30)
lov_annspec_13_dp = ann_spectra_apec_norm(lovisari_srcs, lovisari_srcs.r500*1.3, min_snr=30)

In [ ]:
dpath = "non_paper_figures/density_profiles/svikh_13r500_annspec/"
if not os.path.exists(dpath):
    os.makedirs(dpath+"lovisari")
    os.makedirs(dpath+"locuss") 

lov_svikh_13_annspec_gm = []
with tqdm(desc="Measuring Lovisari Gas Masses", total=len(lov_annspec_13_dp)) as onwards:
    for dp in lov_annspec_13_dp:
        if dp is not None:
            rel_src = lovisari_srcs[dp.src_name]
            # Fit a simplified Vikhlinin density profile
            dp.fit("simple_vikhlinin_dens", progress_bar=False)
            dp.view(save_path=dpath+"lovisari/{}.pdf".format(rel_src.name))
            try:
                # Measure a gas mass within R500
                gm = dp.gas_mass("simple_vikhlinin_dens", rel_src.r500)[0]
                lov_svikh_13_annspec_gm.append(gm)
            except ModelNotAssociatedError:
                lov_svikh_13_annspec_gm.append([np.NaN, np.NaN, np.NaN])
        else:
            lov_svikh_13_annspec_gm.append([np.NaN, np.NaN, np.NaN])
    
        onwards.update(1)

loc_svikh_13_annspec_gm = []
with tqdm(desc="Measuring LoCuSS Gas Masses", total=len(loc_annspec_13_dp)) as onwards:
    for dp in loc_annspec_13_dp:
        if dp is not None:
            rel_src = locuss_srcs[dp.src_name]
            # Fit a simplified Vikhlinin density profile
            dp.fit("simple_vikhlinin_dens", progress_bar=False)
            dp.view(save_path=dpath+"locuss/{}.pdf".format(rel_src.name))
            try:
                # Measure a gas mass within R500
                gm = dp.gas_mass("simple_vikhlinin_dens", rel_src.r500)[0]
                loc_svikh_13_annspec_gm.append(gm)
            except ModelNotAssociatedError:
                loc_svikh_13_annspec_gm.append([np.NaN, np.NaN, np.NaN])
        else:
            loc_svikh_13_annspec_gm.append([np.NaN, np.NaN, np.NaN])
    
        onwards.update(1)

lov_svikh_13_annspec_gm = Quantity(lov_svikh_13_annspec_gm, 'Msun').astype(float)
loc_svikh_13_annspec_gm = Quantity(loc_svikh_13_annspec_gm, 'Msun').astype(float)

In [ ]:
# Setting the y-position and font size of the a and b  labels that are added below the subplots
sublabel_ypos = -0.115
sublabel_fsize = 14

# Set up the matplotlib figure with 2 columns, meaning three subplots
fig, ax_arr = plt.subplots(ncols=2, figsize=(12, 6))

# Iterating through the array of axes objects, setting up the ticks
for ax in ax_arr:
    ax.minorticks_on()
    ax.tick_params(which='both', top=True, right=True, direction='in')

# Reading out the first axis in the array of axes
ax = ax_arr[0]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
lov_lims = find_lims(lov_lit_gms, lov_svikh_13_annspec_gm).value
lov_lims[1] = 5e+14

# Plotting a 1:1 line to help with comparisons
plt.plot(lov_lims, lov_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(lov_lit_gms[:, 0].value, lov_svikh_13_annspec_gm[:, 0].value, 
             xerr=lov_lit_gms[:, 1:].T.value, 
             yerr=lov_svikh_13_annspec_gm[:, 1:].T.value, fmt='x', 
             color=lov_colour, capsize=2, label="Lovisari et al.")

# Labelling axes, setting labels etc.
ax.set_xlabel(r"Lovisari Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(lov_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(lov_lims)
ax.text(0.5, sublabel_ypos, s='a)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

# Reading out the second axis in the array of axes
ax = ax_arr[1]
plt.sca(ax)

# Fetching appropriate limits for comparing the two sets of gas-mass measurements
loc_lims = find_lims(loc_lit_gms, loc_svikh_13_annspec_gm).value

# Plotting a 1:1 line to help with comparisons
plt.plot(loc_lims, loc_lims, color='red', linestyle="dashed", label="1:1")
# Plotting the two datasets with uncertainties
plt.errorbar(loc_lit_gms[:, 0].value, loc_svikh_13_annspec_gm[:, 0].value, 
             xerr=loc_lit_gms[:, 1].value, 
             yerr=loc_svikh_13_annspec_gm[:, 1:].T.value, fmt='x', 
             color=loc_colour, capsize=2, label=r"LoCuSS High-$L_{\rm{X}}$")

# Labelling axes, setting labels etc.
ax.set_xlabel(r"LoCuSS Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_xlim(loc_lims)
ax.set_ylabel("XGA Gas Mass [M$_{\odot}$]", fontsize=15)
ax.set_ylim(loc_lims)
ax.text(0.5, sublabel_ypos, s='b)', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize=sublabel_fsize)
ax.legend(loc='best', fontsize=14)

# Saving and plotting the resulting figure
plt.tight_layout()
plt.show()